## Part 1

In [ ]:
%reset -f

## Part 2 :

In [25]:
%reset -f

In [26]:
import numpy as np
import time

In [27]:
## Construction of the adjacency matrix representing the graph
A = np.zeros((9,9));
A[0,1] = 1
A[0,5] = 1
A[1,2] = 1
A[1,6] = 1
A[2,3] = 1
A[2,7] = 1
A[3,8] = 1
A[4,6] = 1
A[4,7] = 1
A[5,7] = 1
A[5,8] = 1
A[6,8] = 1
A = A + np.transpose(A);

#List that represents the color of the edges in the coloration problem, we start by giving them all the "impostor" color.
joueursCol =[1,1,1,1,1,1,1,1,1]

#List of potential impostors (the alive players that saw player 0 before he got killed)
imp_poten=[1,4,5]

In [28]:
def Solution(A,joueursCol,imp_poten):
    solution = []
    for i in imp_poten:
        joueursCol1 = joueursCol.copy()   #Copying the list of colors so it resets every loop
        for j in range(len(joueursCol)):  #Here, we "colorize" the potential impostor - neighbours, as they can't be impostors 
            if A[i-1,j] == 1 :
                joueursCol1[j]=0
        for h in range(len(joueursCol)): #finally, we create a list of the possible impostors duo, for each "seed" potential impostor
            if joueursCol1[h]==1 and h != i-1:  #owe try to avoid getting tuples such as (1,1)
                solution+= [(i,h+1)]  
                
    #And finally, we print the three possible sets, depending on the seed impostor            
    print(u"The possible impostor set if player 1 killed player 0 is : \n S1 = " + " {" + str(solution[0:6]) + "}")
    print(u"The possible impostor set if player 4 killed player 0 is : \n S2 = " + " {" + str(solution[6:12]) + "}")
    print(u"The possible impostor set if player 5 killed player 0 is : \n S3 = " + " {" + str(solution[12:18]) + "}")
    return(solution[0:6],solution[6:12],solution[12:18])

Solution(A,joueursCol,imp_poten);

The possible impostor set if player 1 killed player 0 is : 
 S1 =  {[(1, 3), (1, 4), (1, 5), (1, 7), (1, 8), (1, 9)]}
The possible impostor set if player 4 killed player 0 is : 
 S2 =  {[(4, 1), (4, 2), (4, 5), (4, 6), (4, 7), (4, 8)]}
The possible impostor set if player 5 killed player 0 is : 
 S3 =  {[(5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 9)]}


## Part 3 :

In [29]:
%reset -f

In [35]:
import numpy as np
import time

In [47]:
# First of all, let's create a dict containing the names of the rooms,
# correspondings to their numbers 
dict_rooms = {1 : "Caféteria", 2 : "Weapons", 3 : "Navigations",
              4 : "O2", 5 : "Shield", 6 : "Communication",
              7 : "Storage", 8 : "Electrical",  9 : "Medbay",
              10 : "Upper E", 11 : "Lower E", 12 : "Reactor",
              13 : "Security", 14 : "Admin"}

# Then, let's create the adjacency matrix for the crewmates map's graph, using the mesure we took on the map
map_crewmates = np.zeros((14,14))    

map_crewmates[0,1] = 5.8
map_crewmates[0,6] = 8.3
map_crewmates[0,8] = 7.15
map_crewmates[0,9] = 9.8
map_crewmates[0,13] = 7.25
map_crewmates[1,2] = 7.15
map_crewmates[1,3] = 4.6
map_crewmates[1,4] = 8.8
map_crewmates[2,3] = 6.6
map_crewmates[2,4] = 7.8
map_crewmates[3,4] = 8.4
map_crewmates[4,5] = 4.1
map_crewmates[4,6] = 6.65
map_crewmates[5,6] = 5.5
map_crewmates[6,7] = 6.85
map_crewmates[6,10] = 9.85
map_crewmates[6,13] = 6.21
map_crewmates[7,10] = 7.7
map_crewmates[8,9] = 7.15
map_crewmates[9,10] = 7.3
map_crewmates[9,11] = 5.55
map_crewmates[9,12] = 5.1
map_crewmates[10,11] = 5.7
map_crewmates[10,12] = 5.2
map_crewmates[11,12] = 4.9

map_impostors = map_crewmates.copy()

map_crewmates += np.transpose(map_crewmates)


### replacing the values, adding the edges representing the new ways of moving using the vents

map_impostors[9,11]=2.8
map_impostors[10,11]=2.8
map_impostors[8,12]=2.6
map_impostors[7,8]=2.3
map_impostors[7,12]=2.5
map_impostors[0,13]=3.6
map_impostors[0,3]=7.1
map_impostors[0,4]=6
map_impostors[0,2]=6.8
map_impostors[2,13]=6
map_impostors[3,13]=6.3
map_impostors[4,13]=5.2
map_impostors[1,13]=3.6
map_impostors[2,4]=2.3
map_impostors[1,2]=2.6

map_impostors += np.transpose(map_impostors)

print("Map crewmates : \n",map_crewmates)
print("\nMap impostors : \n",map_impostors)

Map crewmates : 
 [[0.   5.8  0.   0.   0.   0.   8.3  0.   7.15 9.8  0.   0.   0.   7.25]
 [5.8  0.   7.15 4.6  8.8  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   7.15 0.   6.6  7.8  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   4.6  6.6  0.   8.4  0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   8.8  7.8  8.4  0.   4.1  6.65 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   4.1  0.   5.5  0.   0.   0.   0.   0.   0.   0.  ]
 [8.3  0.   0.   0.   6.65 5.5  0.   6.85 0.   0.   9.85 0.   0.   6.21]
 [0.   0.   0.   0.   0.   0.   6.85 0.   0.   0.   7.7  0.   0.   0.  ]
 [7.15 0.   0.   0.   0.   0.   0.   0.   0.   7.15 0.   0.   0.   0.  ]
 [9.8  0.   0.   0.   0.   0.   0.   0.   7.15 0.   7.3  5.55 5.1  0.  ]
 [0.   0.   0.   0.   0.   0.   9.85 7.7  0.   7.3  0.   5.7  5.2  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   5.55 5.7  0.   4.9  0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   5.1  5.2  4.9  0.   0.  ]
 [7.25 0.   0.   0.   0.   0.   6

In [45]:
###In order to use Dijkstra and Floyd-Warshall, we first create two function, to convert an adjacency matrix in adjacency list, and vice-versa

#### Matrix to adjacency list 
def Mat_to_list(A):
    list_adj = {}     #creation d'un dictionnaire vide
    for i in range(len(A)):
        a=i+1                   # indice in the dict
        list_adj[a] = {}        # creation of one dict by room
        for j in range (len(A)):    # For every item in the matrix, we had the value to the corresponding dict
            if A[i,j] != 0:
                list_adj[a][j+1]=A[i,j] 
    return(list_adj)

B = Mat_to_list(map_crewmates)  # Convert crewmates and impostors graph's adjacency matrix in an adjacency list
C= Mat_to_list(map_impostors)
#print ?

In [38]:
### List to adjacency Matrix
def List_to_Matrix(B):
    a = len(B) 
    mat_adj= np.zeros((a,a))
    for key,value in B.items() :    # For every item in the dict, we get the value and add it in the corresponding spot in the matrix
        for i in value :
            a,b = i
            mat_adj[key-1,a-1]=b
            
    return(mat_adj)

## Dijkstra
def Dijkstra(dg,s):
    infini = sum(sum(dg[sommet][s2]for s2 in dg[sommet]) for sommet in dg) +1  # Create an "infinite" value, that we are sure that no path can even get close to
    s_connu = {s : [0,[s]]}         ## distance from s to s is set to 0
    s_inconnu = {k : [infini, ''] for k in dg if k!=s}  ## distance from s to any other vertice is set to infinite
    for suivant in dg[s]:    #For every direct neighbour of s, we set the distance as being the weight of the path from s to said vertice
        s_inconnu[suivant] = [dg[s][suivant],s]  
    while s_inconnu and any(s_inconnu[k][0]<infini for k in s_inconnu):  #tant qu'il reste des elements dans s_inconnu de distance non infinie
        u = min(s_inconnu, key = s_inconnu.get)  #pick a vertice in set of unknow vertices and set the distance
        longueur_u, precedent_u = s_inconnu[u]
        for v in dg[u]:
            if v in s_inconnu:  #for every unknow vertice connected to u
                d = longueur_u + dg[u][v]  #Check for the path's weight
                if d<s_inconnu[v][0]:       # if it's smaller from the current path
                    s_inconnu[v] = [d,u]    # set it as the new distance/path
        s_connu[u] = [longueur_u,s_connu[precedent_u][1]+[u]]
        del s_inconnu[u]     #delete visited vertice
    return(s_connu)

## Dijkstra applied to every edge
def All_Shortest_Path_D(dg):
    start_time = time.time()   #timer in order to check which algorithm is the fastest
    a = {}
    for j in range (len(dg)):     #simply apply Dijkstra to every single vertice
        a[j+1] = Dijkstra(dg,j+1)
    print(time.time()-start_time)
    return(a)
           
INF = 9999


# Function in order to round up the value in the floyd-warshall distance matrix
def transformation(graph):
    l = len(graph)
    for i in range (l) :
        for j in range(l):
            if graph[i,j]==0:
                graph[i,j]=INF
    graph=np.around(graph,decimals=2)
    return(graph)
      
## Floyd-Warshall
def floyd_warshall(G):
    start_time = time.time()
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))  #distance matrix
    nV=len(G)
    # Adding vertices individually
    for k in range(nV):         
        for i in range(nV):         
            for j in range(nV): 
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])  #Comparing the distances and picking the smallest one
    print(time.time()-start_time)
    return(np.around(distance,decimals=1))

In [48]:
def Exercice3():
    mat = np.zeros((14,14))
    mat1 = np.zeros((14,14))
    mat2=np.zeros((14,14))
    a = All_Shortest_Path_D(B)
    b = All_Shortest_Path_D(C)
    print("Crewmates times :\n")
    for cle,valeurs in a.items():  #For every room
        for key,value in valeurs.items() :  #Print the time it takes to move to another room as a crewmate
             print (u"The shortest path from " +str(cle) + " to " + str(key) +" as a crewmate takes " + str(np.around(value[0],decimals=2)) + " sec")
             mat[cle-1,key-1]=np.around(value[0],decimals=2) #Fill a matrix of distances
    
    print("\n\nImpostors times :\n")
    for cle,valeurs in b.items():       #Same as before but as impostors
        for key,value in valeurs.items() :
             print (u"The shortest path from " +str(dict_rooms[cle]) + " to " + str(dict_rooms[key]) +" as an impostor takes " + str(np.around(value[0],decimals=2))+ " sec")
             mat1[key-1,cle-1]=np.around(value[0],decimals=2) 
    print(mat1)
    for i in range(14):         
        for j in range(i,14):       #Fill the upper side of the matrix with crewmates travelling times to move from a room to another
            mat2[i,j]=mat[i,j]  
        for k in range(0,i+1):      #Fill the lower side of the matrix with impostors travelling times to move from a room to another
            mat2[i,k] = mat1[i,k]
    return(mat2)
    
            #Finally, compare the timings and list the difference of travelling time between crewmates and impostors

def Impostors_Shorter(mat):
    for i in range(len(mat)):
        for j in range(i,len(mat)):
            if mat[i,j]>mat[j,i]:
                print(u"Impostors take " + str(np.around((mat[i,j] - mat[j,i]),decimals =2))+ " secs less than crewmates to reach " + str(dict_rooms[j+1])+ " from " + str(dict_rooms[i+1]))

Exercice3();

0.0009992122650146484
0.0010006427764892578
Crewmates times :

The shortest path from 1 to 1 as a crewmate takes 0 sec
The shortest path from 1 to 2 as a crewmate takes 5.8 sec
The shortest path from 1 to 9 as a crewmate takes 7.15 sec
The shortest path from 1 to 14 as a crewmate takes 7.25 sec
The shortest path from 1 to 7 as a crewmate takes 8.3 sec
The shortest path from 1 to 10 as a crewmate takes 9.8 sec
The shortest path from 1 to 4 as a crewmate takes 10.4 sec
The shortest path from 1 to 3 as a crewmate takes 12.95 sec
The shortest path from 1 to 6 as a crewmate takes 13.8 sec
The shortest path from 1 to 5 as a crewmate takes 14.6 sec
The shortest path from 1 to 13 as a crewmate takes 14.9 sec
The shortest path from 1 to 8 as a crewmate takes 15.15 sec
The shortest path from 1 to 12 as a crewmate takes 15.35 sec
The shortest path from 1 to 11 as a crewmate takes 17.1 sec
The shortest path from 2 to 2 as a crewmate takes 0 sec
The shortest path from 2 to 4 as a crewmate takes 4.6

## Part 4

In [ ]:
%reset -f